In [211]:
import sqlite3
import json
import pandas as pd

In [229]:
class_data = pd.read_json('catalog.json').T

coi_df = class_data['coi_data'].apply(pd.Series)
gpa_df = class_data['gpa_distro'].apply(pd.Series)



In [230]:
class_data

,coi_data,concurrent_courses,course_campus,course_credits,course_description,course_id,course_offered,course_title,department_abbrev,gpa_distro,is_bottleneck,is_gateway,prereq_graph,prereq_string
SWA101,"{'course_coi': 1.7, 'course_level_coi': 0.0, '...",{},seattle,5.0,Introduces the Swahili language and the divers...,SWA 101,None,Basic Swahili,SWA,"[{'count': 0, 'gpa': '0'}, {'count': 0, 'gpa':...",False,False,"{'x': {'edges': {'from': {'8590': 'SWA 101'}, ...",None
SWA102,"{'course_coi': 1.5, 'course_level_coi': 0.0, '...",{},seattle,5.0,Introduces the Swahili language and the divers...,SWA 102,None,Basic Swahili,SWA,"[{'count': 0, 'gpa': '0'}, {'count': 0, 'gpa':...",False,False,"{'x': {'edges': {'from': {'8590': 'SWA 101', '...",SWA 101.
SWA103,"{'course_coi': 1.5, 'course_level_coi': 0.0, '...",{},seattle,5.0,Introduces the Swahili language and the divers...,SWA 103,None,Basic Swahili,SWA,"[{'count': 0, 'gpa': '0'}, {'count': 0, 'gpa':...",True,False,"{'x': {'edges': {'from': {'8591': 'SWA 102', '...",SWA 102.
SWA134,"{'course_coi': None, 'course_level_coi': 0.0, ...",{},seattle,15.0,Introduces the Swahili language and the divers...,SWA 134,S.,Intensive Basic Swahili,SWA,"[{'count': 0, 'gpa': '0'}, {'count': 0, 'gpa':...",False,False,"{'x': {'edges': {'from': {'8593': 'SWA 134'}, ...",None
SWA201,"{'course_coi': None, 'course_level_coi': -0.1,...",{},seattle,5.0,Builds proficiency in the language by speaking...,SWA 201,None,Intermediate Swahili,SWA,"[{'count': 0, 'gpa': '0'}, {'count': 0, 'gpa':...",False,False,"{'x': {'edges': {'from': {'8592': 'SWA 103', '...",either SWA 103 or SWA 134.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HSERV482,"{'course_coi': 1.6, 'course_level_coi': 0.2, '...","{'EPI 320': {'coi_score': 0.5, 'is_bottleneck'...",seattle,4.0,Explores what makes a population healthy or un...,HSERV 482,jointly with G H 482.,The Health of Populations,HSERV,"[{'count': 1, 'gpa': '0'}, {'count': 0, 'gpa':...",False,False,None,None
HSERV488,"{'course_coi': None, 'course_level_coi': 0.2, ...",None,seattle,6.0,None,HSERV 488,None,"Dark Empire: Race, Health, and British Society...",HSERV,[],False,False,None,None
HSERV490,"{'course_coi': None, 'course_level_coi': 0.2, ...",{},seattle,1.0 - 6.0,Explores current or evolving public health pro...,HSERV 490,W.,Advanced Topics,HSERV,"[{'count': 0, 'gpa': '0'}, {'count': 0, 'gpa':...",False,False,None,None
HSERV493,"{'course_coi': None, 'course_level_coi': 0.2, ...",None,seattle,2.0,None,HSERV 493,None,Public Health Uganda,HSERV,[],False,False,None,None


In [231]:
# Merge back into original dataframe (if needed)
class_data = class_data.drop(columns=['coi_data']).join(coi_df)
class_data = class_data.drop(columns=['gpa_distro']).join(gpa_df)


In [232]:
class_data.columns = class_data.columns.astype(str)


In [253]:
class_data['course_id'] = class_data['course_id'].str.replace(" ", "")

In [233]:
def parse_seasons(value):
    value = str(value)  # Convert None to string "None" for processing
    return pd.Series({
        'autumn': 1 if 'A' in value else 0,
        'winter': 1 if 'W' in value else 0,
        'spring': 1 if 'Sp' in value else 0,
        'summer': 1 if 'S' in value and 'Sp' not in value else 0  # Avoid overlap with 'Sp' for spring
    })

# Apply function to create new columns
season_columns = class_data['course_offered'].apply(parse_seasons)
class_data = class_data.join(season_columns)


In [234]:
gpa_df = class_data[[str(i) for i in range(0, 41)]].applymap(lambda x: x['count'] if isinstance(x, dict) else None)
gpa_df.columns = [f"{col}" for col in gpa_df.columns]


class_data = class_data.drop(columns=[str(i) for i in range(0, 41)]).join(gpa_df)


/var/folders/98/0nc594rd7cq2v8bj1t1x697m0000gn/T/ipykernel_64571/962390800.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  gpa_df = class_data[[str(i) for i in range(0, 41)]].applymap(lambda x: x['count'] if isinstance(x, dict) else None)


In [235]:
class_data = class_data.drop(['concurrent_courses', 'prereq_graph'], axis = 1)

In [236]:
class_data.columns

Index(['course_campus', 'course_credits', 'course_description', 'course_id',
       'course_offered', 'course_title', 'department_abbrev', 'is_bottleneck',
       'is_gateway', 'prereq_string', 'course_coi', 'course_level_coi',
       'curric_coi', 'percent_in_range', 'autumn', 'winter', 'spring',
       'summer', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35',
       '36', '37', '38', '39', '40'],
      dtype='object')

In [237]:
import numpy as np

# Extract GPA distribution data
gpa_df = class_data[[str(i) for i in range(0, 41)]]

# Convert column names to GPA values (0.0 to 4.0)
gpa_values = np.array([i / 10 for i in range(41)], dtype = float)  # GPA bins from 0.0 to 4.0

# Compute total number of students in each class
total_counts = gpa_df.sum(axis=1)

# Compute mean GPA for each class
mean_gpa = (gpa_df * gpa_values).sum(axis=1) / total_counts

# Expand data to compute median & standard deviation
expanded_data = gpa_df.apply(lambda row: np.repeat(gpa_values, row.values.astype(int)), axis=1)
median_gpa = expanded_data.apply(np.median)
std_gpa = expanded_data.apply(np.std)

# Compute percentage of 4.0 GPAs
four_point_zero_percent = (gpa_df['40'] / total_counts) * 100  # Column '40' corresponds to GPA 4.0

# Create summary dataframe
summary_df = pd.DataFrame({
    'mean': mean_gpa,
    'median': median_gpa,
    'sd': std_gpa,
    'fourpointo_per': four_point_zero_percent
}).fillna(0)


/var/folders/98/0nc594rd7cq2v8bj1t1x697m0000gn/T/ipykernel_64571/3355881745.py:16: RuntimeWarning: invalid value encountered in cast
  expanded_data = gpa_df.apply(lambda row: np.repeat(gpa_values, row.values.astype(int)), axis=1)
/Users/ethan/dawg_finderAI/dawgfinder-ai/.venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/ethan/dawg_finderAI/dawgfinder-ai/.venv/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/Users/ethan/dawg_finderAI/dawgfinder-ai/.venv/lib/python3.12/site-packages/numpy/_core/_methods.py:223: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/ethan/dawg_finderAI/dawgfinder-ai/.venv/lib/python3.12/site-packages/numpy/_core/_methods.py:181: RuntimeWarning: invalid value encountered in divide

In [238]:
class_data = class_data.merge(summary_df, left_index=True, right_index=True)


In [239]:
class_data

,course_campus,course_credits,course_description,course_id,course_offered,course_title,department_abbrev,is_bottleneck,is_gateway,prereq_string,...,35,36,37,38,39,40,mean,median,sd,fourpointo_per
SWA101,seattle,5.0,Introduces the Swahili language and the divers...,SWA 101,None,Basic Swahili,SWA,False,False,None,...,7.0,9.0,7.0,21.0,15.0,88.0,3.737059,4.0,0.471646,51.764706
SWA102,seattle,5.0,Introduces the Swahili language and the divers...,SWA 102,None,Basic Swahili,SWA,False,False,SWA 101.,...,8.0,6.0,10.0,12.0,18.0,79.0,3.550877,3.9,0.782137,46.198830
SWA103,seattle,5.0,Introduces the Swahili language and the divers...,SWA 103,None,Basic Swahili,SWA,True,False,SWA 102.,...,5.0,4.0,5.0,3.0,9.0,39.0,3.415000,3.8,0.755827,39.000000
SWA134,seattle,15.0,Introduces the Swahili language and the divers...,SWA 134,S.,Intensive Basic Swahili,SWA,False,False,None,...,0.0,0.0,0.0,0.0,1.0,22.0,3.962500,4.0,0.160241,91.666667
SWA201,seattle,5.0,Builds proficiency in the language by speaking...,SWA 201,None,Intermediate Swahili,SWA,False,False,either SWA 103 or SWA 134.,...,0.0,0.0,0.0,0.0,0.0,12.0,4.000000,4.0,0.000000,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HSERV482,seattle,4.0,Explores what makes a population healthy or un...,HSERV 482,jointly with G H 482.,The Health of Populations,HSERV,False,False,None,...,10.0,15.0,35.0,49.0,74.0,35.0,3.709053,3.8,0.459021,14.403292
HSERV488,seattle,6.0,None,HSERV 488,None,"Dark Empire: Race, Health, and British Society...",HSERV,False,False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,0.000000,0.000000
HSERV490,seattle,1.0 - 6.0,Explores current or evolving public health pro...,HSERV 490,W.,Advanced Topics,HSERV,False,False,None,...,0.0,0.0,0.0,0.0,0.0,14.0,4.000000,4.0,0.000000,100.000000
HSERV493,seattle,2.0,None,HSERV 493,None,Public Health Uganda,HSERV,False,False,None,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.0,0.000000,0.000000


In [240]:
# Identify the correct column names (modify based on your actual column names)
numeric_columns = class_data.columns[31:41]  # Adjust if needed
class_data[numeric_columns] = class_data[numeric_columns].apply(pd.to_numeric, errors='coerce')

# String conversion for specific columns
string_columns = [
    "course_campus", "course_credits", "course_description", "course_id",
    "course_offered", "course_title", "department_abbrev", "is_bottleneck",
    "is_gateway", "prereq_string"
]
class_data[string_columns] = class_data[string_columns].astype(str)

# Verify changes
print(class_data.dtypes)


course_campus          object
course_credits         object
course_description     object
course_id              object
course_offered         object
                       ...   
40                    float64
mean                  float64
median                float64
sd                    float64
fourpointo_per        float64
Length: 63, dtype: object


In [241]:
import json

for col in class_data.columns:
    if class_data[col].apply(lambda x: isinstance(x, dict)).any():
        class_data[col] = class_data[col].apply(json.dumps)  # Convert to JSON string


In [242]:
class_data.columns

Index(['course_campus', 'course_credits', 'course_description', 'course_id',
       'course_offered', 'course_title', 'department_abbrev', 'is_bottleneck',
       'is_gateway', 'prereq_string', 'course_coi', 'course_level_coi',
       'curric_coi', 'percent_in_range', 'autumn', 'winter', 'spring',
       'summer', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23',
       '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35',
       '36', '37', '38', '39', '40', 'mean', 'median', 'sd', 'fourpointo_per'],
      dtype='object')

In [243]:
# Connect to the SQLite database
conn = sqlite3.connect('classes.db')
cursor = conn.cursor()

# Define table name
table_name = 'class_data'  # Adjust this to match your actual table name


# Insert DataFrame into the SQLite table
class_data.to_sql(table_name, conn, index=False)  # `replace` ensures old data is cleared

# Commit and close connection
conn.commit()
conn.close()

In [245]:
class_schedule = pd.read_csv('class_schedule.csv')

In [257]:
class_schedule.rename(columns={'Course Name': 'course_id'}, inplace=True)


In [247]:
# Connect to the SQLite database
conn = sqlite3.connect('class_schedule.db')
cursor = conn.cursor()

# Define table name
table_name = 'class_schedule'  # Adjust this to match your actual table name


# Insert DataFrame into the SQLite table
class_schedule.to_sql(table_name, conn, index=False)  # `replace` ensures old data is cleared

# Commit and close connection
conn.commit()
conn.close()

In [258]:
import sqlite3

# Connect to the SQLite database (use the same file for both tables)
db_name = 'classes.db'  # Ensure both tables are in the same database
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# Insert DataFrame into the SQLite table
class_data.to_sql('class_data', conn, index=False, if_exists='replace')  # Replace ensures old data is cleared
class_schedule.to_sql('class_schedule', conn, index=False, if_exists='replace')

# Commit and close connection
conn.commit()
conn.close()
